# NBA Player Performance Trend Analysis
**By Sibhi Sakthivel**

# 1) Introduction

Even the best athletes in the world have an off night eventually, sometimes even a few bad games in a row.

# 2) Research Question

In this study, we are going to be designing, executing, and sharing my findings on trends in NBA player scoring performances. Specifically, we are going to be addressing the question:

**Do sustained departures from a player’s long-term scoring expectation correlate with an increased likelihood of subsequent over- or under-performance relative to that expectation?**

In other words, we are analyzing whether a period of consecutive games with poor scoring performances is correlated with a higher chance of an upcoming game exceeding baseline scoring performance, and vice versa.

# 3) Context, Assumptions, and Confounding Factors

NBA player performance is influenced by several factors, including opponent matchup and teammate availability. These two factors can have a significant impact on a player's efficiency, usage, and minutes played, which can influence how well a player plays and the statistics they record.

Because opponent strength and lineup composition are not randomly assigned, they represent potential confounding variables when analyzing short-term performance trends. Apparent performance “reversion” or “continuation” may reflect changes in these factors rather than intrinsic performance dynamics. For this reason, the analysis proceeds in stages. Initial results examine the raw relationship between sustained performance deviations and subsequent outcomes, followed by conditioned analyses that explicitly account for matchup difficulty and teammate availability to assess robustness.

This analysis is observational and does not attempt to establish causal effects. Instead, it aims to evaluate whether commonly cited performance narratives retain predictive or descriptive value once key contextual factors are considered.

# 3) Data Overview

### Unit of Analysis

The primary unit of analysis in this study is a player–game observation, where each row represents an individual player’s scoring performance in a single NBA game.

### Data Source

The dataset was constructed from the publicly available NBA API, in which we retrieved game-level data including player box score statistics, game context variables, teammate availability, and opponent matchup metrics.

### Data Scope

The analysis focuses on regular-season games from the 2024–25 NBA season, with players required to meet minimum participation thresholds to ensure stable baseline estimates.

For clarity and interpretability, this study focuses exclusively on player scoring performance, measured using points scored per game. While player performance is multi-dimensional, scoring provides a clear, widely understood outcome that aligns with common performance narratives and decision-making contexts.

Additionally, the analysis is conducted at the individual player level, using a single high-usage player as a case study. This design choice allows for detailed examination of performance dynamics without conflating effects across heterogeneous player roles, play styles, and usage profiles.

While this limits the generalizability of results, the framework is designed to be extensible to additional players and performance metrics in future work.

### Data Limitations

Several limitations are inherent in the dataset. Defensive matchup difficulty is represented using team-level and positional defensive metrics, which serve as coarse proxies and do not capture individual on-ball defensive assignments or in-game matchup adjustments.

Teammate availability is approximated using minutes-based thresholds, where high-usage teammates are considered “available” if they play at least a fixed proportion of their season-average minutes. While this approach captures major role-level changes, it may not fully reflect partial usage restrictions, staggered rotations, or in-game context that affects effective player involvement.

In [ ]:
# Load in dataset

# 4) Methodology

### Baseline Scoring Performance

A player’s baseline scoring performance is defined as their season-average points per game across all regular-season games played up to a given point in the season. This baseline is intended to represent a player’s long-term expected scoring output under typical conditions, abstracting away short-term variance and game-to-game noise.

By anchoring performance deviations to a long-term expectation, the analysis focuses on whether sustained departures from expected scoring levels are associated with short-term reversion or continuation in subsequent games.

Rolling-window baselines were considered but not used in order to avoid conflating short-term form with long-term expectation, which is central to the research question.

### Performance Deviation

For each game, a player’s performance deviation is defined as the difference between their observed scoring output in that game and their season-average scoring baseline. This deviation represents how much a player over- or under-performed relative to their long-term expected scoring level.

Positive deviations indicate games in which the player scored more points than their season average, while negative deviations indicate games in which the player scored fewer points than expected. By preserving the direction of the deviation, this definition allows performance to be evaluated not only in terms of magnitude but also in terms of whether a player exceeded or fell short of their baseline expectation.

This measure serves as the fundamental building block for identifying sustained periods of over- or under-performance across consecutive games, which are examined in subsequent sections.

### Sustained Performance Streaks

Each game is first classified as an overperformance or underperformance based on whether the player’s scoring output exceeds or falls below their season-average baseline. Using this classification, a sustained performance streak is defined as a sequence of consecutive games in which the player consistently overperforms or underperforms relative to baseline.

In this study, a streak is considered to occur when a player records at least three consecutive overperforming or underperforming games. The length of the streak corresponds to the number of consecutive games meeting this criterion. This framework allows performance trends to be represented as discrete states that can be used to evaluate how the likelihood of subsequent over- or underperformance changes as streak length increases.

### Outcome Definition

The primary outcome of interest is the player’s scoring performance in the game immediately following a sustained performance streak. For each identified streak, the subsequent game is classified based on whether the player’s scoring output exceeds or falls below their season-average baseline.

This outcome is represented as a binary indicator, where an overperformance corresponds to scoring above baseline and an underperformance corresponds to scoring below baseline. This formulation allows the probability of over- or underperformance in the next game to be estimated conditional on the length and direction of the preceding streak.

### Contextual Adjustments

Opponent difficulty is represented using team-level defensive metrics, which serve as coarse proxies for overall matchup strength. These measures are used to stratify analyses and assess whether observed performance relationships persist across varying levels of opponent defensive quality.

Teammate availability is approximated using minutes-based thresholds. High-usage teammates are considered available in a given game if they play at least a fixed proportion of their season-average minutes. This approach is intended to capture major role-level shifts while remaining interpretable and reproducible.

These contextual variables are not used to redefine performance baselines or streaks, but are incorporated in subsequent analyses to evaluate whether observed performance patterns are robust to changes in game context.

# 5) Analysis

